In [1]:
import numpy as np
import tensorflow as tf
from keras import *
from keras.layers import *
from keras import backend as K
from keras.utils import np_utils 
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import *
from keras.optimizers import SGD
import cv2
import glob
import pydot
import graphviz
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# ZFNet

## ZFNet implemented from scratch with Keras
## based on the paper: Visualizing and Understanding Convolutional Networks

## https://arxiv.org/pdf/1311.2901.pdf


### The data is from a dataset of flower images, with 10 different classes or types of flowers. We will resize the images to be 224x224x3

In [2]:
data = pd.read_csv("data/flower_images/flower_labels.csv")
data["file"] = "data/flower_images/" + data["file"]
X = np.array(data.iloc(1)[0])
y = np.array(data.iloc(1)[1])

In [3]:
def cross_val_split_set(X,portion,y=None):
    '''
    use:
        X = iris.data
        y = iris.target
        X_train, X_test, y_train, y_test = split_set(X,0.1,y) for 10% used as test set
    '''
    X = np.array(X)
    y = np.array(y)
    size = int(X.shape[0]*portion)
    indexlist = np.arange(X.shape[0])
    testinds = np.random.choice(indexlist, size, replace=False)
    traininds = np.array([x for x in range(X.shape[0]) if x not in testinds])  
    if np.all(y == None):
        return X[traininds],X[testinds]
    else:
        return X[traininds],X[testinds],y[traininds],y[testinds]

X_train, X_test, y_train, y_test = cross_val_split_set(X,0.1,y)

In [4]:
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

In [5]:
def preprocess(path,length,width):
    img = cv2.imread(path)
    img = np.asfarray(cv2.resize(img,(length, width)))
    return img/255.0

def read_bulk(paths):
    result = []
    for i in paths:
        result.append(preprocess(i,227,227))
    return np.array(result)

result = read_bulk(X_train)

In [6]:
class ZFNet():
    """
    ZFNet implemented with Keras
    introduced in the paper "Visualizing and Understanding Convolutional Networks"
    https://arxiv.org/pdf/1311.2901.pdf   
    Parameters:
        X: numpy array data matrix 
        y: numpy array of labels, to_categorical changes it to a sparse binary matrix
        weights: name of file that denotes weights to load in
    """
    def __init__(self,X,y,weights="None"):
        
        self.X = X
        self.y = to_categorical(y)
                
        self.weights = weights
        self.model = None
        
        if self.weights == "None":
            self.initialize()
        else:
            self.model = load_model(weights)
        
    def initialize(self):
        
        K.clear_session()
        
        n_outputs = self.y.shape[1]
        height = self.X.shape[1]
        width = self.X.shape[2]
        
        inp = Input(shape=(height,width,3))
        conv1 = Conv2D(96,kernel_size=7,strides=2,border_mode='valid',activation='relu')(inp)
        max1 = MaxPool2D(3,strides=2,border_mode='same')(conv1)
        dropout1 = Dropout(0.5)(max1)
        normal1 = BatchNormalization()(dropout1)
        conv2 = Conv2D(256,kernel_size=5,border_mode='same')(normal1)
        max2 = MaxPool2D(3,strides=2,border_mode='same')(conv2)
        dropout2 = Dropout(0.5)(max2)
        normal2 = BatchNormalization()(dropout2)
        conv3 = Conv2D(512,kernel_size=3,border_mode='same')(normal2)
        conv4 = Conv2D(1024,kernel_size=3,border_mode='same')(conv3)
        conv5 = Conv2D(512,kernel_size=5,border_mode='same')(conv4)
        max3 = MaxPool2D(3,strides=2,border_mode='same')(conv5)
        dropout3 = Dropout(0.5)(max3)
        flatten = Flatten()(dropout3)
        dense1 = Dense(4096,activation="relu")(flatten)
        dropout4 = Dropout(0.5)(dense1)
        dense2 = Dense(4096,activation="relu")(dropout4)
        dropout5 = Dropout(0.5)(dense2)
        dense3 = Dense(n_outputs,activation="softmax")(dropout5)
        dropout6 = Dropout(0.5)(dense3)
        softmax = Softmax(n_outputs)(dropout6)
        
        model = Model(inputs=inp,outputs=softmax)
        model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
        self.model = model
        print(self.model.summary())
        
    def save_picture(self,filename):
        plot_model(self.model, to_file=filename)
        
    def train(self,epochs,save=True):

        self.model.fit(self.X, self.y ,validation_split=0.1, epochs=epochs,verbose=1)
        if save == True:
            self.model.save('saved_models/ZFNet.h5')
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        print('Train Accuracy: %f' % (acc*100))
        
    def predict(self,X):
        
        if len(X.shape) == 3:
            X = X.reshape(1,X.shape[0],X.shape[1],X.shape[2])
        predictions = self.model.predict(X)
        return np.argmax(predictions)
        

In [7]:
result[0].shape

(227, 227, 3)

In [8]:
zf = ZFNet(result,y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 111, 111, 96)      14208     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 56, 56, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0         
__________

In [9]:
zf.save_picture("../model_images/ZFNet.png")

In [10]:
zf.train(1)

Train on 170 samples, validate on 19 samples
Epoch 1/5


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-2a2f9995ccd4>", line 1, in <module>
    zf.train(5)
  File "<ipython-input-6-e4da1d4f63dc>", line 65, in train
    self.model.fit(self.X, self.y ,validation_split=0.1, epochs=epochs,verbose=1)
  File "/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/keras/engine/training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/keras/engine/training.py", line 1235, in _fit_loop
    outs = f(ins_batch)
  File "/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2476, in __call__
    session = get_session()
  File "/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 199

KeyboardInterrupt: 

In [ ]:
zf.predict(result[0])